## Handling Missing Attributes and Rule Induction on Iris Dataset !!

In [1]:
import time
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 200)
from pandas.api.types import is_numeric_dtype
from functools import reduce
from pandas.api.types import is_numeric_dtype
from itertools import groupby

In [2]:
df = pd.read_csv('../data/Iris/Iris-10-lost.csv')

In [3]:
# df = df.drop(['Unnamed: 0', 'sort_col'], axis=1)

In [4]:
df

,sepal_length,sepal_width,petal_length,petal_width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,?,3,1.4,?,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5,?,1.4,0.2,Iris-setosa
5,5.4,3.9,1.7,0.4,Iris-setosa
6,4.6,3.4,1.4,0.3,Iris-setosa
7,5,3.4,1.5,0.2,Iris-setosa
8,4.4,2.9,1.4,?,Iris-setosa
9,4.9,3.1,?,0.1,Iris-setosa


### Filling up the missing attributes with Lost Value Interpretation !!

#### Defining Goal Set !!

In [5]:
df_headers = list(df)
concept = df_headers[-1]
concept

'class'

In [6]:
#all unique concepts
concept_list = df[concept].unique()
concept_list

array(['Iris-setosa', 'Iris-versicolor', 'Iris-viginica'], dtype=object)

In [7]:
#calculating cases by concepts and making sets
U = [] #universal list containing all cases
temp_list = []
goal_list = []
for item in concept_list:
    for index, row in df.iterrows():
        U.append(index+1)
        if row[concept] == item:
            temp_list.append(index)
    goal_list.append(temp_list)
    temp_list = []

In [8]:
print(goal_list,)

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49], [50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99], [100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149]]


#### Building Case List !!

In [9]:
attributes = list(df)
attributes

['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'class']

In [10]:
case_list = []

In [11]:
def discretize(numeric_col):
    print(numeric_col)
    #Sorting the values of numeric column
    sort_col = df[numeric_col].sort_values()
   
    df['sort_col'] = sort_col.values
    point_list = df['sort_col'].unique()
    point_list = list(point_list)
    if '?' in point_list:
        point_list.remove('?')
    print(point_list)
    
    #Finding average between each two points
    avg_list = []
    for i in range(len(point_list)-1):
        avg = (float(point_list[i]) + float(point_list[i+1]))/2
        avg_list.append(round(float(avg),1))
        
    print (avg_list)
    #Performing the discretization and adding the cases
    for i in avg_list:
        case = str(numeric_col) + "," + str(round(float(point_list[0]),1)) + ".." + str(i)
        case2 = str(numeric_col) + "," + str(i) + ".." + str(round(float(point_list[len(point_list)-1]),1))
        case_list.append(case)
        case_list.append(case2)

In [12]:
#Discretization considering upto 2 decimal point
for item in attributes[:-1]:
        discretize(item)

sepal_length
['4.3', '4.4', '4.5', '4.6', '4.7', '4.8', '4.9', '5', '5.1', '5.2', '5.3', '5.4', '5.5', '5.6', '5.7', '5.8', '5.9', '6', '6.1', '6.2', '6.3', '6.4', '6.5', '6.6', '6.7', '6.8', '6.9', '7', '7.1', '7.2', '7.3', '7.4', '7.6', '7.7', '7.9']
[4.3, 4.5, 4.5, 4.7, 4.8, 4.8, 5.0, 5.0, 5.2, 5.2, 5.3, 5.5, 5.5, 5.7, 5.8, 5.8, 6.0, 6.0, 6.2, 6.2, 6.3, 6.5, 6.5, 6.7, 6.8, 6.8, 7.0, 7.0, 7.2, 7.2, 7.3, 7.5, 7.7, 7.8]
sepal_width
['2.2', '2.3', '2.4', '2.5', '2.6', '2.7', '2.8', '2.9', '3', '3.1', '3.2', '3.3', '3.4', '3.5', '3.6', '3.7', '3.8', '3.9', '4', '4.1', '4.2', '4.4']
[2.2, 2.3, 2.5, 2.5, 2.7, 2.8, 2.8, 3.0, 3.0, 3.2, 3.2, 3.3, 3.5, 3.5, 3.7, 3.8, 3.8, 4.0, 4.0, 4.2, 4.3]
petal_length
['1', '1.1', '1.2', '1.3', '1.4', '1.5', '1.6', '1.7', '1.9', '3', '3.3', '3.5', '3.6', '3.7', '3.8', '3.9', '4', '4.1', '4.2', '4.3', '4.4', '4.5', '4.6', '4.7', '4.8', '4.9', '5', '5.1', '5.2', '5.3', '5.4', '5.5', '5.6', '5.7', '5.8', '5.9', '6', '6.1', '6.3', '6.4', '6.6', '6.7', '6.9']
[1

In [13]:
case_list

['sepal_length,4.3..4.3',
 'sepal_length,4.3..7.9',
 'sepal_length,4.3..4.5',
 'sepal_length,4.5..7.9',
 'sepal_length,4.3..4.5',
 'sepal_length,4.5..7.9',
 'sepal_length,4.3..4.7',
 'sepal_length,4.7..7.9',
 'sepal_length,4.3..4.8',
 'sepal_length,4.8..7.9',
 'sepal_length,4.3..4.8',
 'sepal_length,4.8..7.9',
 'sepal_length,4.3..5.0',
 'sepal_length,5.0..7.9',
 'sepal_length,4.3..5.0',
 'sepal_length,5.0..7.9',
 'sepal_length,4.3..5.2',
 'sepal_length,5.2..7.9',
 'sepal_length,4.3..5.2',
 'sepal_length,5.2..7.9',
 'sepal_length,4.3..5.3',
 'sepal_length,5.3..7.9',
 'sepal_length,4.3..5.5',
 'sepal_length,5.5..7.9',
 'sepal_length,4.3..5.5',
 'sepal_length,5.5..7.9',
 'sepal_length,4.3..5.7',
 'sepal_length,5.7..7.9',
 'sepal_length,4.3..5.8',
 'sepal_length,5.8..7.9',
 'sepal_length,4.3..5.8',
 'sepal_length,5.8..7.9',
 'sepal_length,4.3..6.0',
 'sepal_length,6.0..7.9',
 'sepal_length,4.3..6.0',
 'sepal_length,6.0..7.9',
 'sepal_length,4.3..6.2',
 'sepal_length,6.2..7.9',
 'sepal_leng

#### Handle Missing Attributes !!

In [14]:
#For Lost values, that case wouldn't be included in any block

In [15]:
df_headers = list(df)
concept = df_headers[-2]
concept_list = df[concept].unique()

In [16]:
concept_list

array(['Iris-setosa', 'Iris-versicolor', 'Iris-viginica'], dtype=object)

In [17]:
#calculating cases by concepts and making sets
temp_list = []
goal_list = []
for item in concept_list:
    for index, row in df.iterrows():
        if row[concept] == item:
            temp_list.append(index+1)
    goal_list.append(temp_list)
    temp_list = []

In [18]:
print(goal_list,)

[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50], [51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100], [101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150]]


In [19]:
temp_list = []
att_val_list = []
for item in case_list:
    a,b = item.split(",") #a = attribute and b = value
  
    if ".." in b:
        start,end = b.split("..")
        for index, row in df.iterrows():
            if row[a] is not '?':
                if float(row[a]) >= float(start) and float(row[a]) <= float(end):
                    temp_list.append(index+1)
        
        att_val_list.append(temp_list)
        temp_list = []
        
    else:
        for index, row in df.iterrows():
            if type(row[a]) == list:
                tmp_list = row[a]
                for case in tmp_list:
                    if float(case) == float(b):
                        temp_list.append(index+1)
                        
            if float(row[a]) == float(b):
                temp_list.append(index+1)
       
        att_val_list.append(temp_list)
        temp_list = []

In [20]:
#Print two evenly distributed list side by side
fmt = '%-8s%-20s%s'

for i, (case, att_val) in enumerate(zip(case_list, att_val_list)):
    print(fmt % (i, case, att_val))

0       sepal_length,4.3..4.3[14]
1       sepal_length,4.3..7.9[1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 44, 45, 46, 47, 48, 49, 50, 51, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 71, 72, 73, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 115, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 134, 135, 136, 137, 138, 139, 140, 141, 143, 144, 145, 146, 147, 148, 149, 150]
2       sepal_length,4.3..4.5[9, 14, 39, 42]
3       sepal_length,4.5..7.9[1, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 15, 16, 17, 18, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 40, 41, 42, 44, 45, 46, 47, 48, 49, 50, 51, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 71, 72, 73, 75, 76, 77, 7

In [21]:
#Creating data for case and att-value list
data = {'Cases': case_list, 'att_val': att_val_list}

In [22]:
df2 = pd.DataFrame(data)

In [23]:
#Cases and corresponding att-value pairs
df2

,Cases,att_val
0,"sepal_length,4.3..4.3",[14]
1,"sepal_length,4.3..7.9","[1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1..."
2,"sepal_length,4.3..4.5","[9, 14, 39, 42]"
3,"sepal_length,4.5..7.9","[1, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 15, 16, ..."
4,"sepal_length,4.3..4.5","[9, 14, 39, 42]"
5,"sepal_length,4.5..7.9","[1, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 15, 16, ..."
6,"sepal_length,4.3..4.7","[3, 4, 7, 9, 14, 23, 30, 39, 42, 48]"
7,"sepal_length,4.7..7.9","[1, 3, 5, 6, 8, 10, 11, 12, 13, 15, 16, 17, 18..."
8,"sepal_length,4.3..4.8","[3, 4, 7, 9, 12, 13, 14, 23, 25, 30, 31, 39, 4..."
9,"sepal_length,4.8..7.9","[1, 5, 6, 8, 10, 11, 12, 13, 15, 16, 17, 18, 2..."


### Start of Lower/Upper & Probabilistic Approximations !!

#### Building Chacteristic Set !!

In [24]:
attributes = list(df)
del attributes[-1]

In [25]:
attributes

['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'class']

In [26]:
case_list = []
#Loop through all the attributes except last 2 - Concept and sort_col
for item in attributes[:-1]:
    print(item)
    
    #check for non numeric columns
    if not is_numeric_dtype(df[item]):
        temp = df[item].unique()
        for i in temp:
            if i == '?' or i == '-':
                continue
            else:
                case = item + "," + i
                case_list.append(case)

sepal_length
sepal_width
petal_length
petal_width


In [27]:
case_list

['sepal_length,5.1',
 'sepal_length,4.7',
 'sepal_length,4.6',
 'sepal_length,5',
 'sepal_length,5.4',
 'sepal_length,4.4',
 'sepal_length,4.9',
 'sepal_length,4.8',
 'sepal_length,4.3',
 'sepal_length,5.8',
 'sepal_length,5.7',
 'sepal_length,5.2',
 'sepal_length,5.5',
 'sepal_length,4.5',
 'sepal_length,5.3',
 'sepal_length,7',
 'sepal_length,6.9',
 'sepal_length,6.5',
 'sepal_length,6.3',
 'sepal_length,6.6',
 'sepal_length,5.9',
 'sepal_length,6',
 'sepal_length,6.1',
 'sepal_length,5.6',
 'sepal_length,6.7',
 'sepal_length,6.2',
 'sepal_length,6.4',
 'sepal_length,6.8',
 'sepal_length,7.1',
 'sepal_length,7.6',
 'sepal_length,7.3',
 'sepal_length,7.2',
 'sepal_length,7.7',
 'sepal_length,7.4',
 'sepal_length,7.9',
 'sepal_width,3.5',
 'sepal_width,3',
 'sepal_width,3.2',
 'sepal_width,3.1',
 'sepal_width,3.9',
 'sepal_width,3.4',
 'sepal_width,2.9',
 'sepal_width,3.7',
 'sepal_width,4',
 'sepal_width,4.4',
 'sepal_width,3.8',
 'sepal_width,3.6',
 'sepal_width,3.3',
 'sepal_width,4

In [28]:
temp_list = []
att_val_list = []
for item in case_list:
    a,b = item.split(",") #a = attribute and b = value
    if ".." in b:
        start,end = b.split("..")
        for index, row in df.iterrows():
            if row[a] >= int(start) and row[a] <= int(end):
                temp_list.append(index+1)
        print(temp_list)
        att_val_list.append(temp_list)
        temp_list = []
        
    else:
        for index, row in df.iterrows():
            if type(row[a]) == list:
                tmp_list = row[a]
                for case in tmp_list:
                    if case == b:
                        temp_list.append(index+1)
                        
            if row[a] == b:
                temp_list.append(index+1)
        print(temp_list)
        att_val_list.append(temp_list)
        temp_list = []

[1, 18, 20, 22, 24, 40, 45, 47, 99]
[3, 30]
[4, 7, 23, 48]
[5, 8, 26, 27, 36, 41, 44, 50, 61, 94]
[6, 11, 17, 21, 32, 85]
[9, 39]
[10, 35, 38, 58, 107]
[12, 13, 25, 31, 46]
[14]
[15, 68, 83, 93, 102, 115, 143]
[16, 56, 80, 97, 100]
[28, 29, 33, 60]
[34, 37, 54, 81, 82, 90, 91]
[42]
[49]
[51]
[53, 121, 140]
[55, 105, 111, 117, 148]
[57, 73, 88, 101, 104, 124, 134, 137, 147]
[59, 76]
[62, 71, 150]
[63, 79, 84, 86, 120, 139]
[64, 72, 92, 128, 135]
[65, 67, 89, 95, 122]
[66, 78, 87, 109, 125, 141, 145, 146]
[69, 98, 127, 149]
[75, 112, 129, 138]
[77, 113, 144]
[103]
[106]
[108]
[110, 126, 130]
[118, 119, 123, 136]
[131]
[132]
[1, 18, 37, 41, 44]
[2, 13, 14, 26, 39, 46, 62, 78, 89, 92, 105, 106, 113, 128, 130, 136, 139, 146, 148, 150]
[3, 30, 36, 43, 48, 51, 52, 71, 111, 121, 126, 144]
[4, 10, 31, 35, 38, 53, 66, 87, 138, 140, 141, 142]
[6, 17]
[7, 8, 12, 21, 25, 27, 29, 32, 86, 137, 149]
[9, 59, 64, 65, 75, 79, 97, 98, 104, 108]
[11, 22, 49]
[15]
[16]
[19, 20, 118, 132]
[23, 110]
[24, 50, 

In [29]:
#Print two evenly distributed list side by side
fmt = '%-8s%-20s%s'

for i, (case, att_val) in enumerate(zip(case_list, att_val_list)):
    print(fmt % (i, case, att_val))

0       sepal_length,5.1    [1, 18, 20, 22, 24, 40, 45, 47, 99]
1       sepal_length,4.7    [3, 30]
2       sepal_length,4.6    [4, 7, 23, 48]
3       sepal_length,5      [5, 8, 26, 27, 36, 41, 44, 50, 61, 94]
4       sepal_length,5.4    [6, 11, 17, 21, 32, 85]
5       sepal_length,4.4    [9, 39]
6       sepal_length,4.9    [10, 35, 38, 58, 107]
7       sepal_length,4.8    [12, 13, 25, 31, 46]
8       sepal_length,4.3    [14]
9       sepal_length,5.8    [15, 68, 83, 93, 102, 115, 143]
10      sepal_length,5.7    [16, 56, 80, 97, 100]
11      sepal_length,5.2    [28, 29, 33, 60]
12      sepal_length,5.5    [34, 37, 54, 81, 82, 90, 91]
13      sepal_length,4.5    [42]
14      sepal_length,5.3    [49]
15      sepal_length,7      [51]
16      sepal_length,6.9    [53, 121, 140]
17      sepal_length,6.5    [55, 105, 111, 117, 148]
18      sepal_length,6.3    [57, 73, 88, 101, 104, 124, 134, 137, 147]
19      sepal_length,6.6    [59, 76]
20      sepal_length,5.9    [62, 71, 150]
21      sepal

In [30]:
U = set(U)
print(U,)

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150}


In [31]:
# Creating dictionary combining case_list and att_val list
block = dict(zip(case_list, att_val_list))
block

{'sepal_length,5.1': [1, 18, 20, 22, 24, 40, 45, 47, 99],
 'sepal_length,4.7': [3, 30],
 'sepal_length,4.6': [4, 7, 23, 48],
 'sepal_length,5': [5, 8, 26, 27, 36, 41, 44, 50, 61, 94],
 'sepal_length,5.4': [6, 11, 17, 21, 32, 85],
 'sepal_length,4.4': [9, 39],
 'sepal_length,4.9': [10, 35, 38, 58, 107],
 'sepal_length,4.8': [12, 13, 25, 31, 46],
 'sepal_length,4.3': [14],
 'sepal_length,5.8': [15, 68, 83, 93, 102, 115, 143],
 'sepal_length,5.7': [16, 56, 80, 97, 100],
 'sepal_length,5.2': [28, 29, 33, 60],
 'sepal_length,5.5': [34, 37, 54, 81, 82, 90, 91],
 'sepal_length,4.5': [42],
 'sepal_length,5.3': [49],
 'sepal_length,7': [51],
 'sepal_length,6.9': [53, 121, 140],
 'sepal_length,6.5': [55, 105, 111, 117, 148],
 'sepal_length,6.3': [57, 73, 88, 101, 104, 124, 134, 137, 147],
 'sepal_length,6.6': [59, 76],
 'sepal_length,5.9': [62, 71, 150],
 'sepal_length,6': [63, 79, 84, 86, 120, 139],
 'sepal_length,6.1': [64, 72, 92, 128, 135],
 'sepal_length,5.6': [65, 67, 89, 95, 122],
 'sepal

In [32]:
dic = {}
for index, row in df.iterrows():
    tmp_set = set()
    final_union = []
    char_list = []
    char_list_2 = []
    final_union_set = []
  
    for cols in attributes[:-1]:
        #If the value for corresponding attribute is a list then create all of the att-value pairs
        if type(df.loc[index,cols]) == list:
            print("When values are list") #########
            for item in df.loc[index,cols]:
                block_key = cols + "," + item
                char_list.append(block_key) #char_list has all att-val cases
                print(char_list) #########
                
            union_set = set()
            #Compute union of att-concept value case
            for item in char_list:
                union_set = union_set.union(set(block[item]))
            
            print("Union Set: ", union_set) #########
            final_union.append(union_set)
        
        else:
            print("When value is single") #########
            block_key = cols + "," + str(df.loc[index,cols])
            char_list_2.append(block_key) #char_list_2 has all single cases
            print(char_list_2) ##########
   
    #Compute instersection for this current row for Characteristics set
    
    print("final_union: ", final_union) ########
    if len(final_union):
        final_union_set = list(reduce(set.intersection, [set(item) for item in final_union]))
        
    
    print("Final Union Set: ", final_union_set) ##########
    
    for item in char_list_2:
        if item in block:
            print("When item is found as key in the block") ########
            if tmp_set == set():
                #Copy over the current set elements to B
                for i in range(len(block[item])):
                    tmp_set.add(block[item][i])
                    
            tmp_set = tmp_set.intersection(set(block[item]))
            print(tmp_set) ########
        
        #If item not in block
        else:
            print("When item is not found as key in the block") ########
            print(tmp_set) ########
            print(U) #########
            if tmp_set == set():
                tmp_set = U
                
            tmp_set = tmp_set.intersection(U)
            print(tmp_set) ########
    
    print("Final tmp_set: ", tmp_set) #########
    final_union_set = set(final_union_set)
    
    if final_union_set == set():
        tmp_set = tmp_set
    else:
        tmp_set = tmp_set.intersection(final_union_set)
        
    print("This is the final value: ", tmp_set) ###########
        
    key = ('K_%d' % (index+1))
    print(key) #########
    dic[key] = tmp_set
    print("\n") ##########

When value is single
['sepal_length,5.1']
When value is single
['sepal_length,5.1', 'sepal_width,3.5']
When value is single
['sepal_length,5.1', 'sepal_width,3.5', 'petal_length,1.4']
When value is single
['sepal_length,5.1', 'sepal_width,3.5', 'petal_length,1.4', 'petal_width,0.2']
final_union:  []
Final Union Set:  []
When item is found as key in the block
{1, 99, 40, 45, 47, 18, 20, 22, 24}
When item is found as key in the block
{1, 18}
When item is found as key in the block
{1, 18}
When item is found as key in the block
{1}
Final tmp_set:  {1}
This is the final value:  {1}
K_1


When value is single
['sepal_length,?']
When value is single
['sepal_length,?', 'sepal_width,3']
When value is single
['sepal_length,?', 'sepal_width,3', 'petal_length,1.4']
When value is single
['sepal_length,?', 'sepal_width,3', 'petal_length,1.4', 'petal_width,?']
final_union:  []
Final Union Set:  []
When item is not found as key in the block
set()
{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16,

Final tmp_set:  {26}
This is the final value:  {26}
K_26


When value is single
['sepal_length,5']
When value is single
['sepal_length,5', 'sepal_width,3.4']
When value is single
['sepal_length,5', 'sepal_width,3.4', 'petal_length,1.6']
When value is single
['sepal_length,5', 'sepal_width,3.4', 'petal_length,1.6', 'petal_width,?']
final_union:  []
Final Union Set:  []
When item is found as key in the block
{36, 5, 8, 41, 44, 50, 26, 27, 61, 94}
When item is found as key in the block
{8, 27}
When item is found as key in the block
{27}
When item is not found as key in the block
{27}
{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104,

['sepal_length,5.3', 'sepal_width,3.7']
When value is single
['sepal_length,5.3', 'sepal_width,3.7', 'petal_length,1.5']
When value is single
['sepal_length,5.3', 'sepal_width,3.7', 'petal_length,1.5', 'petal_width,?']
final_union:  []
Final Union Set:  []
When item is found as key in the block
{49}
When item is found as key in the block
{49}
When item is found as key in the block
{49}
When item is not found as key in the block
{49}
{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 

{59}
When item is found as key in the block
{59}
When item is found as key in the block
{59}
Final tmp_set:  {59}
This is the final value:  {59}
K_59


When value is single
['sepal_length,5.2']
When value is single
['sepal_length,5.2', 'sepal_width,2.7']
When value is single
['sepal_length,5.2', 'sepal_width,2.7', 'petal_length,3.9']
When value is single
['sepal_length,5.2', 'sepal_width,2.7', 'petal_length,3.9', 'petal_width,?']
final_union:  []
Final Union Set:  []
When item is found as key in the block
{33, 28, 29, 60}
When item is found as key in the block
{60}
When item is found as key in the block
{60}
When item is not found as key in the block
{60}
{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 

When value is single
['sepal_length,6', 'sepal_width,3.4']
When value is single
['sepal_length,6', 'sepal_width,3.4', 'petal_length,4.5']
When value is single
['sepal_length,6', 'sepal_width,3.4', 'petal_length,4.5', 'petal_width,1.6']
final_union:  []
Final Union Set:  []
When item is found as key in the block
{139, 79, 84, 86, 120, 63}
When item is found as key in the block
{86}
When item is found as key in the block
{86}
When item is found as key in the block
{86}
Final tmp_set:  {86}
This is the final value:  {86}
K_86


When value is single
['sepal_length,6.7']
When value is single
['sepal_length,6.7', 'sepal_width,3.1']
When value is single
['sepal_length,6.7', 'sepal_width,3.1', 'petal_length,4.7']
When value is single
['sepal_length,6.7', 'sepal_width,3.1', 'petal_length,4.7', 'petal_width,1.5']
final_union:  []
Final Union Set:  []
When item is found as key in the block
{66, 109, 78, 141, 145, 146, 87, 125}
When item is found as key in the block
{66, 141, 87}
When item is foun

{104}
Final tmp_set:  {104}
This is the final value:  {104}
K_104


When value is single
['sepal_length,6.5']
When value is single
['sepal_length,6.5', 'sepal_width,3']
When value is single
['sepal_length,6.5', 'sepal_width,3', 'petal_length,?']
When value is single
['sepal_length,6.5', 'sepal_width,3', 'petal_length,?', 'petal_width,2.2']
final_union:  []
Final Union Set:  []
When item is found as key in the block
{105, 111, 148, 117, 55}
When item is found as key in the block
{105, 148}
When item is not found as key in the block
{105, 148}
{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112,

When value is single
['sepal_length,6.9', 'sepal_width,3.2', 'petal_length,5.7']
When value is single
['sepal_length,6.9', 'sepal_width,3.2', 'petal_length,5.7', 'petal_width,2.3']
final_union:  []
Final Union Set:  []
When item is found as key in the block
{121, 140, 53}
When item is found as key in the block
{121}
When item is found as key in the block
{121}
When item is found as key in the block
{121}
Final tmp_set:  {121}
This is the final value:  {121}
K_121


When value is single
['sepal_length,5.6']
When value is single
['sepal_length,5.6', 'sepal_width,2.8']
When value is single
['sepal_length,5.6', 'sepal_width,2.8', 'petal_length,4.9']
When value is single
['sepal_length,5.6', 'sepal_width,2.8', 'petal_length,4.9', 'petal_width,2']
final_union:  []
Final Union Set:  []
When item is found as key in the block
{65, 67, 89, 122, 95}
When item is found as key in the block
{122}
When item is found as key in the block
{122}
When item is found as key in the block
{122}
Final tmp_set:

This is the final value:  {141}
K_141


When value is single
['sepal_length,?']
When value is single
['sepal_length,?', 'sepal_width,3.1']
When value is single
['sepal_length,?', 'sepal_width,3.1', 'petal_length,5.1']
When value is single
['sepal_length,?', 'sepal_width,3.1', 'petal_length,5.1', 'petal_width,2.3']
final_union:  []
Final Union Set:  []
When item is not found as key in the block
set()
{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141,

In [33]:
dic

{'K_1': {1},
 'K_2': {2, 13, 46},
 'K_3': {3},
 'K_4': {4},
 'K_5': {5, 50},
 'K_6': {6},
 'K_7': {7},
 'K_8': {8},
 'K_9': {9},
 'K_10': {10, 35, 38},
 'K_11': {11},
 'K_12': {12, 25},
 'K_13': {13},
 'K_14': {14},
 'K_15': {15},
 'K_16': {16},
 'K_17': {17},
 'K_18': {18},
 'K_19': {19},
 'K_20': {20},
 'K_21': {21},
 'K_22': {22},
 'K_23': {23},
 'K_24': {24},
 'K_25': {25},
 'K_26': {26},
 'K_27': {27},
 'K_28': {28},
 'K_29': {29},
 'K_30': {30},
 'K_31': {31},
 'K_32': {32},
 'K_33': {33},
 'K_34': {34},
 'K_35': {35, 38},
 'K_36': {36},
 'K_37': {37},
 'K_38': {35, 38},
 'K_39': {39},
 'K_40': {40},
 'K_41': {41},
 'K_42': {42},
 'K_43': {3, 43},
 'K_44': {44},
 'K_45': {45},
 'K_46': {46},
 'K_47': {47},
 'K_48': {48},
 'K_49': {49},
 'K_50': {50},
 'K_51': {51},
 'K_52': {52},
 'K_53': {53},
 'K_54': {54, 90},
 'K_55': {55},
 'K_56': {56},
 'K_57': {57},
 'K_58': {58},
 'K_59': {59},
 'K_60': {60},
 'K_61': {61},
 'K_62': {62},
 'K_63': {63},
 'K_64': {64},
 'K_65': {65},
 'K_

#### Lower Approximation !!

In [34]:
def lowerApproximation(charac_set,concept):
    #set to contain lower approximations
    lower = set()
    
    #Check for each element of the concept
    for item in concept:
        key = 'K_%d' % (item)
        set_value = charac_set[key]
  
        if set_value.issubset(set(concept)):
            lower = lower.union(set_value)
          
    return lower

In [35]:
lower_approximations = {}
for item in goal_list:
    #Key is the string converted list so as to add as dictionary key
    lower_approximations[str(item)] = lowerApproximation(dic,item)

In [36]:
lower_goal_list=list(lower_approximations.values())
print(lower_goal_list)

[{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50}, {51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100}, {128, 129, 130, 131, 132, 133, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127}]


#### Upper Approximation !!

In [37]:
def upperApproximation(charac_set,concept):
    #set to contain lower approximations
    upper = set()
    
    #Check for each element of the concept
    for item in concept:
        key = 'K_%d' % (item)
        set_value = charac_set[key]
        upper = upper.union(set_value)
          
    return upper

In [38]:
upper_approximations = {}
for item in goal_list:
    #Key is the string converted list so as to add as dictionary key
    upper_approximations[str(item)] = upperApproximation(dic,item)

In [39]:
upper_goal_list=list(upper_approximations.values())
print(upper_goal_list)

[{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50}, {51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100}, {128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 55, 73, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127}]


#### Middle Approximations !!

In [40]:
first_column = list(dic.keys())

In [41]:
second_column = list(dic.values())

In [42]:
prob_approx = pd.DataFrame(
    {'charset_name': first_column,
     'charset_value': second_column
    })

In [43]:
prob_approx

,charset_name,charset_value
0,K_1,{1}
1,K_2,"{2, 13, 46}"
2,K_3,{3}
3,K_4,{4}
4,K_5,"{50, 5}"
5,K_6,{6}
6,K_7,{7}
7,K_8,{8}
8,K_9,{9}
9,K_10,"{10, 35, 38}"


In [44]:
def probabilisticApproximation(concept):
    prob = []
    for index, row in prob_approx.iterrows():
        if len(row['charset_value']) != 0:
            probability_conditional = len(row['charset_value'].intersection(set(concept))) / len(row['charset_value'])
        else:
            probability_conditional = 0.0
            
        prob.append(round(probability_conditional,2))
    return prob

In [45]:
def findMiddleApprox(concept):
    probapprox = set()
    cond_prob = probabilisticApproximation(concept) #Need to put the goal
    prob_approx['cond_probability'] = cond_prob
    for index, row in prob_approx.iterrows():
        part1,part2 = row['charset_name'].split("_")
       
        if row['cond_probability'] >= 0.50:
            if int(part2) in concept:
#                 print(row['charset_value'])
                probapprox = probapprox.union(row['charset_value'])
    return probapprox   

In [46]:
middle_approximations = {}
for item in goal_list:
    print(item)
    #Key is the string converted list so as to add as dictionary key
    middle_approximations[str(item)] = findMiddleApprox(item)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50]
[51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100]
[101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150]


In [47]:
middle_goal_list=list(middle_approximations.values())
print(middle_goal_list)

[{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50}, {51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100}, {128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 55, 73, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127}]


In [48]:
df2

,Cases,att_val
0,"sepal_length,4.3..4.3",[14]
1,"sepal_length,4.3..7.9","[1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1..."
2,"sepal_length,4.3..4.5","[9, 14, 39, 42]"
3,"sepal_length,4.5..7.9","[1, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 15, 16, ..."
4,"sepal_length,4.3..4.5","[9, 14, 39, 42]"
5,"sepal_length,4.5..7.9","[1, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 15, 16, ..."
6,"sepal_length,4.3..4.7","[3, 4, 7, 9, 14, 23, 30, 39, 42, 48]"
7,"sepal_length,4.7..7.9","[1, 3, 5, 6, 8, 10, 11, 12, 13, 15, 16, 17, 18..."
8,"sepal_length,4.3..4.8","[3, 4, 7, 9, 12, 13, 14, 23, 25, 30, 31, 39, 4..."
9,"sepal_length,4.8..7.9","[1, 5, 6, 8, 10, 11, 12, 13, 15, 16, 17, 18, 2..."


In [49]:
df3=df2

In [50]:
import pylab as pl

In [51]:
def findGoalIntersect(goal):
    goalIntersect = []
    
    for index, row in df3.iterrows():
        #List containing intersection of (a,v) pairs and goal
        goalIntersect.append(set(row['att_val']).intersection(set(goal)))
          
    #Check if goal_intersect column exists
    if 'goal_intersect' in df3:
        df3['goal_intersect'] = goalIntersect
    else:
        #Insert new column with the recent iteration
        df3.insert(2, 'goal_intersect', goalIntersect)

In [52]:
def findCases(df3):
    case_to_be = []
    #Find the cases with maximum goal coverage
    m = max(df3['goal_intersect'], key=len)
    possible_cases = [i for i, j in enumerate(df3['goal_intersect'].tolist()) if j == m]
    
    #Index of the case covering max goal and having min no. of elements
    new_df = df3.iloc[possible_cases,:]
    
    m1 = min(new_df['att_val'], key=len)
    
    for index,row in new_df.iterrows():
        if row['att_val'] == m1:
            case_to_be.append(index)
   
    return case_to_be[0]

In [53]:
def combineInterval(test_condition):
    
    test_num = [] #This will contain the conditions having interval
    test_str = [] #This will contain the conditions having no interval
    
    #Loop through to seprate contions having intervals and no intervals
    for item in test_condition:
        if ".." in item:
            test_num.append(item)
        else:
            test_str.append(item)
   
    #Group the conditions having interval based on same attributes
    grouped = [list(g) for k, g in groupby(test_num, lambda s: s.partition(',')[0])]
    
    final_list = []
    
    #Actually combining the intervals
    for list1 in grouped:
        greatest = 0
        smallest = 0
        for item in list1:
            part1,part2 = item.split(",")
            start,stop = part2.split("..")
            start = float(start)
            stop = float(stop)

            if greatest == 0 and smallest == 0:
                greatest = start
                smallest = stop

            if start > greatest:
                greatest = float(start)

            if stop < smallest:
                smallest = float(stop)

        con_tmp = part1 + "," + str(greatest) + ".." + str(smallest)
        final_list.append(con_tmp)  
            
    actual_condition = final_list + test_str
    
    return actual_condition

In [54]:
def dropCondition(condition,current_goal):
    
    for item in range(0,len(condition)):
        temp_att_val = []
        temp_cond = condition.copy() #use list.copy() as equal operator simply copies over the reference
        temp_cond.remove(condition[item])
       
        #temp_cond contains the elements after removing the current element
        for i in temp_cond:
            if i is not None:
                location = df3.index[df3['Cases'] == i].tolist()
                element = df3['att_val'].loc[location[0]]
                temp_att_val.append(set(element))
          
        #temp_att_val contains the actual value set of the corresponding cases
        #Find the intersection if the set has more than one element, otherwise no need
        if len(temp_att_val) > 1:
            intersection = set.intersection(*temp_att_val)
        
            #if the set still remains a subset of the original goal after removing current element
            #then set the current element to None as we want to drop this later
            if intersection.issubset(current_goal):
                condition[item] = None
    
    condition = [x for x in condition if x is not None]
   
    return condition   

In [55]:
def stepAlgo(df3,selected_case,current_goal,B,condition,concept_curr):
    
    rule_set = []
    original_goal = current_goal.copy()
    
    
    while current_goal != set():
        #Check if the selected case is a subset of the current goal
        #List of current case
        A = df3['att_val'].loc[selected_case] 

        if B == set():
            #Copy over the current set elements to B
            for i in range(len(A)):
                B.add(A[i])

        #Elements of intersection of current and previous set
        A = set(A).intersection(B)
        B = A.copy()
        
        print("Intersection set : ",A)
        #Check if intersecting elements are subset of Goal
        if A.issubset(current_goal):
          
            #Current goal is updated after discarding the already covered goal by new rule
            if len(A) != 0:
                current_goal = set(current_goal) - A
            else:
                current_goal = set()
            

            #Extract the current case
            curr_case = df3['Cases'].loc[selected_case]
            #Add the conditions of a Rule
            condition.append(curr_case)
            
            #Check for possibility of dropping conditions
            if len(condition) > 1:
                condition = dropCondition(condition,original_goal)
            
            #Combine the interval
            if len(condition) > 1:
                condition = combineInterval(condition)
            

            #Join conditions
            cond = ""
            for item in condition:
                cond = cond + "(" + str(item) + ")" + " & "

            cond = cond[:-2] + "->"
            rule = cond + " (" + concept + "," + concept_curr + ")"
            rule_set.append(rule)
            
            #Reset everythng and continue for covering rest of the goal
            condition = []
            B = set()
            findGoalIntersect(current_goal)
            selected_case = findCases(df3)


        #If not a subset of current goal
        else:
            print("NOT")
            #Assign empty set for the selected case for next iteration
            df3['goal_intersect'].loc[selected_case] = set()
            print(df3['Cases'].loc[selected_case], " Set to be NULL")
           
            #Extract the current case
            curr_case = df3['Cases'].loc[selected_case]
            #Add the case to the condition list
            condition.append(curr_case)
            
            #Check for Range overlapping of the remaining cases
            if ".." in curr_case:
                second_part = curr_case.split(',')[1]
                start = float(second_part.split('..')[0])
                end = float(second_part.split('..')[1])
                for index, row in df3.iterrows():
                    if ".." in row['Cases']:
                        part2 = row['Cases'].split(',')[1]
                        start1 = float(part2.split('..')[0])
                        end1 = float(part2.split('..')[1])

                        #Assign blank set for cases with overlapping ranges
                        if set((pl.frange(start,end))).issubset(pl.frange(start1,end1)) == True:
                            if start == start1 and end <= end1:
                                row['goal_intersect'] = set()
                                print(row['Cases'], "also set to be NULL")
            
           
            selected_case = findCases(df3)
           
                        
    return rule_set

In [56]:
#concept_list and goal_list has 1:1 mapping
final_rules = []
start_time = time.time()

#Running algorithm for all the goals - Lower Approximation/concepts
for i in range(0,len(middle_goal_list)):
    findGoalIntersect(list(middle_goal_list[i]))
    condition = []
    B = set()
    selected_case = findCases(df3)
    
    rule_set = stepAlgo(df3,selected_case,middle_goal_list[i],B,condition,concept_list[i])
    final_rules.append(rule_set)
    
elapsed_time = time.time() - start_time

Intersection set :  {1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 143, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 44, 45, 46, 47, 48, 49, 50, 54, 56, 58, 60, 61, 65, 67, 68, 80, 81, 82, 83, 85, 89, 90, 91, 93, 94, 95, 97, 99, 100, 102, 107, 115, 122}
NOT
sepal_length,4.3..5.8  Set to be NULL
sepal_length,4.3..7.9 also set to be NULL
sepal_length,4.3..5.8 also set to be NULL
sepal_length,4.3..5.8 also set to be NULL
sepal_length,4.3..6.0 also set to be NULL
sepal_length,4.3..6.0 also set to be NULL
sepal_length,4.3..6.2 also set to be NULL
sepal_length,4.3..6.2 also set to be NULL
sepal_length,4.3..6.3 also set to be NULL
sepal_length,4.3..6.5 also set to be NULL
sepal_length,4.3..6.5 also set to be NULL
sepal_length,4.3..6.7 also set to be NULL
sepal_length,4.3..6.8 also set to be NULL
sepal_length,4.3..6.8 also set to be NULL
sepal_length,4.3..7.0 also set to be NULL
sepal_length,4.3..7.0 also set to be NULL
sepal_length,

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:86: MatplotlibDeprecationWarning: numpy.arange


 also set to be NULL
sepal_width,2.5..4.4 also set to be NULL
petal_length,2.5..6.9 also set to be NULL
Intersection set :  {128, 1, 2, 3, 4, 130, 132, 7, 8, 136, 10, 11, 12, 13, 14, 137, 138, 139, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 148, 29, 30, 31, 32, 35, 36, 37, 38, 39, 41, 43, 44, 46, 48, 49, 50, 51, 52, 53, 62, 140, 66, 141, 71, 142, 78, 144, 86, 87, 89, 145, 92, 146, 101, 105, 106, 110, 111, 149, 113, 150, 118, 121, 126}
NOT
sepal_width,2.2..3.8  Set to be NULL
sepal_width,2.2..4.4 also set to be NULL
sepal_width,2.2..3.8 also set to be NULL
sepal_width,2.2..3.8 also set to be NULL
sepal_width,2.2..4.0 also set to be NULL
sepal_width,2.2..4.0 also set to be NULL
sepal_width,2.2..4.2 also set to be NULL
sepal_width,2.2..4.3 also set to be NULL
Intersection set :  {128, 1, 2, 3, 4, 130, 132, 7, 8, 136, 10, 11, 12, 13, 14, 137, 138, 139, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 148, 29, 30, 31, 32, 35, 36, 37, 38, 39, 41, 43, 44, 46, 48, 49, 50, 51, 52, 53, 62, 140, 66, 141, 71,

Intersection set :  {128, 139, 142, 143, 146, 147, 148, 150, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 78, 79, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 102, 107, 111, 114, 115, 120, 122, 124, 127}
NOT
petal_length,1.2..6.9  Set to be NULL
petal_length,1.2..6.9 also set to be NULL
Intersection set :  {128, 139, 142, 143, 146, 147, 148, 150, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 78, 79, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 102, 107, 111, 114, 115, 120, 122, 124, 127}
NOT
petal_length,1.1..6.9  Set to be NULL
petal_length,1.1..6.9 also set to be NULL
petal_length,1.1..6.9 also set to be NULL
Intersection set :  {128, 139, 142, 143, 146, 147, 148, 150, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 78, 79, 81, 82, 83, 84, 85,

Intersection set :  {128, 129, 130, 131, 133, 135, 136, 138, 139, 140, 141, 142, 143, 144, 146, 147, 148, 150, 51, 52, 53, 55, 56, 58, 59, 60, 62, 64, 65, 66, 68, 70, 71, 72, 73, 74, 75, 78, 79, 81, 82, 84, 87, 89, 90, 91, 92, 93, 94, 95, 97, 98, 99, 100, 102, 104, 106, 107, 108, 109, 111, 113, 114, 119, 121, 122, 123, 124, 127}
NOT
petal_length,1.8..6.9  Set to be NULL
petal_length,1.8..6.9 also set to be NULL
Intersection set :  {128, 129, 130, 131, 133, 135, 136, 138, 139, 140, 141, 142, 143, 144, 146, 147, 148, 150, 51, 52, 53, 55, 56, 58, 59, 60, 62, 64, 65, 66, 68, 70, 71, 72, 73, 74, 75, 78, 79, 81, 82, 84, 87, 89, 90, 91, 92, 93, 94, 95, 97, 98, 99, 100, 102, 104, 106, 107, 108, 109, 111, 113, 114, 119, 121, 122, 123, 124, 127}
NOT
petal_length,1.6..6.9  Set to be NULL
petal_length,1.6..6.9 also set to be NULL
petal_length,1.6..6.9 also set to be NULL
Intersection set :  {128, 139, 142, 143, 146, 147, 148, 150, 51, 52, 53, 55, 56, 58, 59, 60, 62, 64, 65, 66, 68, 70, 71, 72, 73,

...

Intersection set :  {1, 5, 6, 8, 10, 11, 12, 13, 15, 16, 17, 18, 20, 21, 22, 24, 25, 26, 27, 28, 29, 31, 32, 33, 34, 35, 36, 37, 38, 40, 41, 44, 45, 46, 47, 49, 50, 51, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 71, 72, 73, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 115, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 134, 135, 136, 137, 138, 139, 140, 141, 143, 144, 145, 146, 147, 148, 149, 150}
NOT
sepal_length,4.8..7.9  Set to be NULL
sepal_length,4.8..7.9 also set to be NULL
sepal_length,4.8..7.9 also set to be NULL
Intersection set :  {1, 5, 6, 8, 10, 11, 12, 13, 15, 16, 17, 18, 20, 21, 22, 24, 25, 26, 27, 28, 29, 31, 32, 33, 34, 35, 36, 37, 38, 40, 41, 44, 45, 46, 47, 49, 50, 51, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 71, 72, 73, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 8

sepal_length,4.3..7.7 also set to be NULL
sepal_length,4.3..7.8 also set to be NULL
petal_length,4.3..6.9 also set to be NULL
Intersection set :  {128, 129, 130, 131, 135, 136, 137, 138, 139, 140, 143, 144, 145, 146, 147, 148, 149, 150, 53, 57, 64, 67, 69, 71, 73, 79, 84, 86, 87, 92, 101, 102, 103, 104, 106, 108, 109, 111, 112, 113, 115, 118, 119, 120, 121, 122, 123, 124, 127}
NOT
petal_length,4.2..6.9  Set to be NULL
petal_length,4.2..6.9 also set to be NULL
petal_length,4.2..6.9 also set to be NULL
Intersection set :  {128, 129, 130, 131, 135, 136, 137, 138, 139, 140, 143, 144, 145, 146, 147, 148, 149, 150, 53, 57, 64, 67, 69, 71, 73, 79, 84, 86, 87, 92, 101, 102, 103, 104, 106, 108, 109, 111, 112, 113, 115, 118, 119, 120, 121, 122, 123, 124, 127}
NOT
petal_length,4.0..6.9  Set to be NULL
petal_length,4.0..6.9 also set to be NULL
petal_length,4.0..6.9 also set to be NULL
Intersection set :  {128, 129, 130, 131, 135, 136, 137, 138, 139, 140, 143, 144, 145, 146, 147, 148, 149, 150, 53,

Intersection set :  {1, 5, 6, 8, 10, 11, 12, 13, 15, 16, 17, 18, 20, 21, 22, 24, 25, 26, 27, 28, 29, 31, 32, 33, 34, 35, 36, 37, 38, 40, 41, 44, 45, 46, 47, 49, 50, 51, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 71, 72, 73, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 115, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 134, 135, 136, 137, 138, 139, 140, 141, 143, 144, 145, 146, 147, 148, 149, 150}
NOT
sepal_length,4.8..7.9  Set to be NULL
sepal_length,4.8..7.9 also set to be NULL
sepal_length,4.8..7.9 also set to be NULL
Intersection set :  {1, 5, 6, 8, 10, 11, 12, 13, 15, 16, 17, 18, 20, 21, 22, 24, 25, 26, 27, 28, 29, 31, 32, 33, 34, 35, 36, 37, 38, 40, 41, 44, 45, 46, 47, 49, 50, 51, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 71, 72, 73, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 8

petal_length,3.7..6.9 also set to be NULL
Intersection set :  {128, 129, 130, 131, 135, 136, 137, 138, 139, 140, 143, 144, 145, 146, 147, 148, 149, 150, 53, 57, 64, 67, 69, 71, 73, 79, 84, 86, 87, 92, 101, 102, 103, 104, 106, 108, 109, 111, 112, 113, 115, 118, 119, 120, 121, 122, 123, 124, 127}
NOT
petal_length,3.4..6.9  Set to be NULL
petal_length,3.4..6.9 also set to be NULL
Intersection set :  {128, 129, 130, 131, 135, 136, 137, 138, 139, 140, 143, 144, 145, 146, 147, 148, 149, 150, 53, 57, 64, 67, 69, 71, 73, 79, 84, 86, 87, 92, 101, 102, 103, 104, 106, 108, 109, 111, 112, 113, 115, 118, 119, 120, 121, 122, 123, 124, 127}
NOT
petal_length,3.5..6.9  Set to be NULL
petal_length,3.5..6.9 also set to be NULL
Intersection set :  {128, 129, 130, 131, 135, 136, 137, 138, 139, 140, 143, 144, 145, 146, 147, 148, 149, 150, 53, 57, 64, 67, 69, 71, 73, 79, 84, 86, 87, 92, 101, 102, 103, 104, 106, 108, 109, 111, 112, 113, 115, 118, 119, 120, 121, 122, 123, 124, 127}
NOT
petal_length,3.1..6.9  S

petal_width,1.9..2.5 also set to be NULL
petal_width,1.9..2.5 also set to be NULL
Intersection set :  {129, 140, 111, 113, 147, 148}
NOT
petal_width,0.1..2.2  Set to be NULL
petal_width,0.1..2.2 also set to be NULL
petal_width,0.1..2.2 also set to be NULL
petal_width,0.1..2.3 also set to be NULL
petal_width,0.1..2.5 also set to be NULL
Intersection set :  {129, 147, 148, 111}
NOT
sepal_length,4.3..6.7  Set to be NULL
sepal_length,4.3..7.9 also set to be NULL
sepal_length,4.3..6.7 also set to be NULL
sepal_length,4.3..6.8 also set to be NULL
sepal_length,4.3..6.8 also set to be NULL
sepal_length,4.3..7.0 also set to be NULL
sepal_length,4.3..7.0 also set to be NULL
sepal_length,4.3..7.2 also set to be NULL
sepal_length,4.3..7.2 also set to be NULL
sepal_length,4.3..7.3 also set to be NULL
sepal_length,4.3..7.5 also set to be NULL
sepal_length,4.3..7.7 also set to be NULL
sepal_length,4.3..7.8 also set to be NULL
petal_length,4.3..6.9 also set to be NULL
Intersection set :  {129, 148, 11

In [57]:
print("Time to run the algorithm (for Middle Approximation): ", round(elapsed_time, 3), "Sec")
print("\n")
print(*final_rules, sep='\n')

with open('../data/Iris/Rules/Iris_10_lost_middle.txt', 'w') as f:
    for item in final_rules:
        f.write("%s\n" % item)

Time to run the algorithm (for Middle Approximation):  10.199 Sec


['(sepal_length,4.3..5.8) & (petal_length,1.0..2.5) -> (class,Iris-setosa)', '(petal_length,1.0..1.8) & (sepal_length,4.8..7.9) -> (class,Iris-setosa)']
['(sepal_length,5.0..7.9) & (petal_length,3.7..4.7) -> (class,Iris-versicolor)', '(petal_length,4.5..6.9) & (petal_width,0.1..1.2) -> (class,Iris-versicolor)']
['(sepal_length,6.0..7.9) & (petal_width,1.8..2.5) -> (class,Iris-viginica)', '(petal_length,5.2..6.9) & (sepal_length,6.5..7.9) -> (class,Iris-viginica)']


In [58]:
#concept_list and goal_list has 1:1 mapping
final_rules = []
start_time = time.time()

#Running algorithm for all the goals - Lower Approximation/concepts
for i in range(0,len(lower_goal_list)):
    findGoalIntersect(list(lower_goal_list[i]))
    condition = []
    B = set()
    selected_case = findCases(df3)
    
    rule_set = stepAlgo(df3,selected_case,lower_goal_list[i],B,condition,concept_list[i])
    final_rules.append(rule_set)
    
elapsed_time = time.time() - start_time

Intersection set :  {1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 143, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 44, 45, 46, 47, 48, 49, 50, 54, 56, 58, 60, 61, 65, 67, 68, 80, 81, 82, 83, 85, 89, 90, 91, 93, 94, 95, 97, 99, 100, 102, 107, 115, 122}
NOT
sepal_length,4.3..5.8  Set to be NULL
sepal_length,4.3..7.9 also set to be NULL
sepal_length,4.3..5.8 also set to be NULL
sepal_length,4.3..5.8 also set to be NULL
sepal_length,4.3..6.0 also set to be NULL
sepal_length,4.3..6.0 also set to be NULL
sepal_length,4.3..6.2 also set to be NULL
sepal_length,4.3..6.2 also set to be NULL
sepal_length,4.3..6.3 also set to be NULL
sepal_length,4.3..6.5 also set to be NULL
sepal_length,4.3..6.5 also set to be NULL
sepal_length,4.3..6.7 also set to be NULL
sepal_length,4.3..6.8 also set to be NULL
sepal_length,4.3..6.8 also set to be NULL
sepal_length,4.3..7.0 also set to be NULL
sepal_length,4.3..7.0 also set to be NULL
sepal_length,

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:86: MatplotlibDeprecationWarning: numpy.arange


 also set to be NULL
sepal_width,2.2..3.5 also set to be NULL
sepal_width,2.2..3.5 also set to be NULL
sepal_width,2.2..3.7 also set to be NULL
sepal_width,2.2..3.8 also set to be NULL
sepal_width,2.2..3.8 also set to be NULL
sepal_width,2.2..4.0 also set to be NULL
sepal_width,2.2..4.0 also set to be NULL
sepal_width,2.2..4.2 also set to be NULL
sepal_width,2.2..4.3 also set to be NULL
Intersection set :  {1, 3, 4, 7, 8, 10, 12, 13, 14, 18, 21, 25, 26, 29, 30, 31, 35, 37, 38, 39, 41, 43, 46, 48, 50}
NOT
petal_width,0.1..0.3  Set to be NULL
petal_width,0.1..0.3 also set to be NULL
petal_width,0.1..0.5 also set to be NULL
petal_width,0.1..0.6 also set to be NULL
petal_width,0.1..0.8 also set to be NULL
petal_width,0.1..1.1 also set to be NULL
petal_width,0.1..1.1 also set to be NULL
petal_width,0.1..1.2 also set to be NULL
petal_width,0.1..1.4 also set to be NULL
petal_width,0.1..1.4 also set to be NULL
petal_width,0.1..1.6 also set to be NULL
petal_width,0.1..1.6 also set to be NULL
pe

petal_length,1.0..5.2 also set to be NULL
petal_length,1.0..5.2 also set to be NULL
petal_length,1.0..5.3 also set to be NULL
petal_length,1.0..5.5 also set to be NULL
petal_length,1.0..5.5 also set to be NULL
petal_length,1.0..5.7 also set to be NULL
petal_length,1.0..5.8 also set to be NULL
petal_length,1.0..5.8 also set to be NULL
petal_length,1.0..6.0 also set to be NULL
petal_length,1.0..6.0 also set to be NULL
petal_length,1.0..6.2 also set to be NULL
petal_length,1.0..6.3 also set to be NULL
petal_length,1.0..6.5 also set to be NULL
petal_length,1.0..6.7 also set to be NULL
petal_length,1.0..6.8 also set to be NULL
Intersection set :  {128, 139, 142, 143, 146, 147, 148, 150, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 78, 79, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 102, 107, 111, 114, 115, 120, 122, 124, 127}
NOT
petal_length,1.4..6.9  Set to be NULL
petal_length,1.4..6.9 also se

Intersection set :  {2, 3, 4, 9, 10, 13, 14, 26, 30, 31, 35, 36, 38, 39, 42, 43, 46, 48, 51, 52, 53, 55, 56, 58, 59, 60, 62, 63, 64, 65, 66, 68, 69, 70, 71, 72, 73, 74, 75, 77, 78, 79, 80, 81, 82, 84, 87, 89, 90, 91, 92, 93, 94, 95, 97, 98, 99, 100, 102, 104, 105, 106, 107, 108, 109, 111, 113, 114, 119, 120, 121, 122, 123, 124, 126, 127, 128, 129, 130, 131, 133, 135, 136, 138, 139, 140, 141, 142, 143, 144, 146, 147, 148, 150}
NOT
sepal_width,2.2..3.2  Set to be NULL
sepal_width,2.2..4.4 also set to be NULL
sepal_width,2.2..3.2 also set to be NULL
sepal_width,2.2..3.2 also set to be NULL
sepal_width,2.2..3.3 also set to be NULL
sepal_width,2.2..3.5 also set to be NULL
sepal_width,2.2..3.5 also set to be NULL
sepal_width,2.2..3.7 also set to be NULL
sepal_width,2.2..3.8 also set to be NULL
sepal_width,2.2..3.8 also set to be NULL
sepal_width,2.2..4.0 also set to be NULL
sepal_width,2.2..4.0 also set to be NULL
sepal_width,2.2..4.2 also set to be NULL
sepal_width,2.2..4.3 also set to be N

...

Intersection set :  {128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 142, 143, 144, 145, 146, 147, 148, 149, 150, 52, 53, 57, 62, 64, 66, 67, 69, 71, 73, 76, 77, 79, 84, 86, 87, 92, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 118, 119, 120, 121, 122, 123, 124, 126, 127}
NOT
petal_width,1.4..2.5  Set to be NULL
petal_length,1.4..6.9 also set to be NULL
petal_length,1.4..6.9 also set to be NULL
petal_width,1.4..2.5 also set to be NULL
petal_width,1.4..2.5 also set to be NULL
Intersection set :  {128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 142, 143, 144, 145, 146, 147, 148, 149, 150, 52, 53, 57, 62, 64, 66, 67, 69, 71, 73, 76, 77, 79, 84, 86, 87, 92, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 118, 119, 120, 121, 122, 123, 124, 126, 127}
NOT
petal_width,1.2..2.5  Set to be NULL
petal_length,1.2..6.9 also set to be NULL
petal_width,1.2..2.5 also set to be NULL
Intersection set :  {

petal_length,1.6..6.9 also set to be NULL
petal_length,1.6..6.9 also set to be NULL
Intersection set :  {128, 129, 130, 131, 132, 135, 136, 137, 138, 139, 140, 143, 144, 145, 146, 147, 148, 149, 150, 53, 57, 64, 67, 69, 71, 73, 79, 84, 86, 87, 92, 101, 102, 103, 104, 106, 107, 108, 109, 111, 112, 113, 115, 118, 119, 120, 121, 122, 123, 124, 127}
NOT
sepal_length,4.8..7.9  Set to be NULL
sepal_length,4.8..7.9 also set to be NULL
sepal_length,4.8..7.9 also set to be NULL
Intersection set :  {128, 129, 130, 131, 132, 135, 136, 137, 138, 139, 140, 143, 144, 145, 146, 147, 148, 149, 150, 53, 57, 64, 67, 69, 71, 73, 79, 84, 86, 87, 92, 101, 102, 103, 104, 106, 107, 108, 109, 111, 112, 113, 115, 118, 119, 120, 121, 122, 123, 124, 127}
NOT
sepal_length,4.7..7.9  Set to be NULL
sepal_length,4.7..7.9 also set to be NULL
Intersection set :  {128, 129, 130, 131, 132, 135, 136, 137, 138, 139, 140, 143, 144, 145, 146, 147, 148, 149, 150, 53, 71, 73, 84, 101, 102, 103, 104, 106, 108, 109, 111, 112, 1

petal_width,0.5..2.5 also set to be NULL
Intersection set :  {128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 142, 143, 144, 145, 146, 147, 148, 149, 150, 52, 53, 57, 62, 64, 66, 67, 69, 71, 73, 76, 77, 79, 84, 86, 87, 92, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 118, 119, 120, 121, 122, 123, 124, 126, 127}
NOT
petal_width,0.3..2.5  Set to be NULL
petal_width,0.3..2.5 also set to be NULL
Intersection set :  {128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 142, 143, 144, 145, 146, 147, 148, 149, 150, 52, 53, 57, 62, 64, 66, 67, 69, 71, 73, 76, 77, 79, 84, 86, 87, 92, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 118, 119, 120, 121, 122, 123, 124, 126, 127}
NOT
petal_width,0.2..2.5  Set to be NULL
petal_width,0.2..2.5 also set to be NULL
petal_width,0.2..2.5 also set to be NULL
Intersection set :  {128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 142, 143, 144, 145, 

Intersection set :  {128, 129, 130, 131, 132, 136, 137, 138, 139, 140, 143, 144, 145, 146, 147, 148, 149, 150, 71, 84, 101, 102, 103, 104, 106, 108, 109, 111, 112, 113, 115, 118, 121, 122, 123, 124, 127}
NOT
petal_width,1.6..2.5  Set to be NULL
petal_length,1.6..6.9 also set to be NULL
petal_length,1.6..6.9 also set to be NULL
petal_width,1.6..2.5 also set to be NULL
petal_width,1.6..2.5 also set to be NULL
Intersection set :  {128, 129, 130, 131, 132, 136, 137, 138, 139, 140, 143, 144, 145, 146, 147, 148, 149, 150, 71, 84, 101, 102, 103, 104, 106, 108, 109, 111, 112, 113, 115, 118, 121, 122, 123, 124, 127}
NOT
sepal_length,5.5..7.9  Set to be NULL
sepal_length,5.5..7.9 also set to be NULL
sepal_length,5.5..7.9 also set to be NULL
Intersection set :  {128, 129, 130, 131, 132, 136, 137, 138, 139, 140, 143, 144, 145, 146, 147, 148, 149, 150, 71, 84, 101, 102, 103, 104, 106, 108, 109, 111, 112, 113, 115, 118, 121, 122, 123, 124, 127}
NOT
sepal_length,5.3..7.9  Set to be NULL
sepal_length,

petal_width,2.0..2.5 also set to be NULL
Intersection set :  {129, 148, 111}
NOT
sepal_length,4.3..6.7  Set to be NULL
sepal_length,4.3..7.9 also set to be NULL
sepal_length,4.3..6.7 also set to be NULL
sepal_length,4.3..6.8 also set to be NULL
sepal_length,4.3..6.8 also set to be NULL
sepal_length,4.3..7.0 also set to be NULL
sepal_length,4.3..7.0 also set to be NULL
sepal_length,4.3..7.2 also set to be NULL
sepal_length,4.3..7.2 also set to be NULL
sepal_length,4.3..7.3 also set to be NULL
sepal_length,4.3..7.5 also set to be NULL
sepal_length,4.3..7.7 also set to be NULL
sepal_length,4.3..7.8 also set to be NULL
petal_length,4.3..6.9 also set to be NULL
Intersection set :  {129}
NOT
petal_length,5.3..6.9  Set to be NULL
sepal_length,5.3..7.9 also set to be NULL
petal_length,5.3..6.9 also set to be NULL
Intersection set :  {129}
NOT
petal_length,5.2..6.9  Set to be NULL
sepal_length,5.2..7.9 also set to be NULL
sepal_length,5.2..7.9 also set to be NULL
petal_length,5.2..6.9 also set 

In [59]:
print("Time to run the algorithm (for Lower Approximation): ", round(elapsed_time, 3), "Sec")
print("\n")
print(*final_rules, sep='\n')

with open('../data/Iris/Rules/Iris_10_lost_lower.txt', 'w') as f:
    for item in final_rules:
        f.write("%s\n" % item)

Time to run the algorithm (for Lower Approximation):  10.033 Sec


['(sepal_length,4.3..5.8) & (petal_length,1.0..2.5) -> (class,Iris-setosa)', '(petal_length,1.0..1.8) & (sepal_length,4.8..7.9) -> (class,Iris-setosa)']
['(sepal_length,5.0..7.9) & (petal_length,3.7..4.7) -> (class,Iris-versicolor)', '(petal_length,4.5..6.9) & (petal_width,0.1..1.2) -> (class,Iris-versicolor)']
['(petal_width,1.8..2.5) & (petal_length,5.0..6.9) -> (class,Iris-viginica)', '(petal_length,5.2..6.9) & (sepal_width,3.0..4.4) -> (class,Iris-viginica)']


In [60]:
#concept_list and goal_list has 1:1 mapping
final_rules = []
start_time = time.time()

#Running algorithm for all the goals - Lower Approximation/concepts
for i in range(0,len(upper_goal_list)):
    findGoalIntersect(list(upper_goal_list[i]))
    condition = []
    B = set()
    selected_case = findCases(df3)
    
    rule_set = stepAlgo(df3,selected_case,upper_goal_list[i],B,condition,concept_list[i])
    final_rules.append(rule_set)
    
elapsed_time = time.time() - start_time

Intersection set :  {1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 143, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 44, 45, 46, 47, 48, 49, 50, 54, 56, 58, 60, 61, 65, 67, 68, 80, 81, 82, 83, 85, 89, 90, 91, 93, 94, 95, 97, 99, 100, 102, 107, 115, 122}
NOT
sepal_length,4.3..5.8  Set to be NULL
sepal_length,4.3..7.9 also set to be NULL
sepal_length,4.3..5.8 also set to be NULL
sepal_length,4.3..5.8 also set to be NULL
sepal_length,4.3..6.0 also set to be NULL
sepal_length,4.3..6.0 also set to be NULL
sepal_length,4.3..6.2 also set to be NULL
sepal_length,4.3..6.2 also set to be NULL
sepal_length,4.3..6.3 also set to be NULL
sepal_length,4.3..6.5 also set to be NULL
sepal_length,4.3..6.5 also set to be NULL
sepal_length,4.3..6.7 also set to be NULL
sepal_length,4.3..6.8 also set to be NULL
sepal_length,4.3..6.8 also set to be NULL
sepal_length,4.3..7.0 also set to be NULL
sepal_length,4.3..7.0 also set to be NULL
sepal_length,

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:86: MatplotlibDeprecationWarning: numpy.arange


 {128, 1, 2, 3, 4, 130, 132, 7, 8, 136, 10, 11, 12, 13, 14, 137, 138, 139, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 148, 29, 30, 31, 32, 35, 36, 37, 38, 39, 41, 43, 44, 46, 48, 49, 50, 51, 52, 53, 62, 140, 66, 141, 71, 142, 78, 144, 86, 87, 89, 145, 92, 146, 101, 105, 106, 110, 111, 149, 113, 150, 118, 121, 126}
NOT
sepal_width,2.3..4.4  Set to be NULL
sepal_width,2.3..4.4 also set to be NULL
Intersection set :  {128, 1, 2, 3, 4, 130, 7, 8, 136, 10, 137, 12, 13, 14, 138, 139, 140, 18, 141, 142, 21, 144, 145, 24, 25, 26, 27, 148, 29, 30, 31, 32, 35, 36, 37, 38, 39, 41, 43, 44, 46, 48, 50, 51, 52, 53, 62, 66, 71, 78, 86, 87, 89, 92, 146, 101, 105, 106, 149, 111, 113, 150, 121, 126}
NOT
sepal_width,2.2..3.5  Set to be NULL
sepal_width,2.2..4.4 also set to be NULL
sepal_width,2.2..3.5 also set to be NULL
sepal_width,2.2..3.5 also set to be NULL
sepal_width,2.2..3.7 also set to be NULL
sepal_width,2.2..3.8 also set to be NULL
sepal_width,2.2..3.8 also set to be NULL
sepal_width,2.2..4.0 also

Intersection set :  {51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 78, 79, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 106, 107, 108, 109, 111, 112, 113, 114, 115, 116, 118, 119, 120, 121, 122, 123, 124, 125, 127, 128, 129, 130, 131, 132, 133, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150}
NOT
petal_length,1.6..6.9  Set to be NULL
petal_length,1.6..6.9 also set to be NULL
petal_length,1.6..6.9 also set to be NULL
Intersection set :  {128, 139, 142, 143, 146, 147, 148, 150, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 78, 79, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 102, 107, 111, 114, 115, 120, 122, 124, 127}
NOT
petal_length,1.0..5.2  Set to be NULL
petal_length,1.0..5.2 also set to be NULL
petal_length,1.0..5.2 also set to be NULL
petal_leng

Intersection set :  {2, 3, 4, 9, 10, 13, 14, 26, 30, 31, 35, 36, 38, 39, 42, 43, 46, 48, 51, 52, 53, 55, 56, 58, 59, 60, 62, 63, 64, 65, 66, 68, 69, 70, 71, 72, 73, 74, 75, 77, 78, 79, 80, 81, 82, 84, 87, 89, 90, 91, 92, 93, 94, 95, 97, 98, 99, 100, 102, 104, 105, 106, 107, 108, 109, 111, 113, 114, 119, 120, 121, 122, 123, 124, 126, 127, 128, 129, 130, 131, 133, 135, 136, 138, 139, 140, 141, 142, 143, 144, 146, 147, 148, 150}
NOT
sepal_width,2.2..3.2  Set to be NULL
sepal_width,2.2..4.4 also set to be NULL
sepal_width,2.2..3.2 also set to be NULL
sepal_width,2.2..3.2 also set to be NULL
sepal_width,2.2..3.3 also set to be NULL
sepal_width,2.2..3.5 also set to be NULL
sepal_width,2.2..3.5 also set to be NULL
sepal_width,2.2..3.7 also set to be NULL
sepal_width,2.2..3.8 also set to be NULL
sepal_width,2.2..3.8 also set to be NULL
sepal_width,2.2..4.0 also set to be NULL
sepal_width,2.2..4.0 also set to be NULL
sepal_width,2.2..4.2 also set to be NULL
sepal_width,2.2..4.3 also set to be N

...

Intersection set :  {128, 129, 130, 131, 132, 134, 135, 136, 137, 138, 139, 140, 143, 144, 145, 146, 147, 148, 149, 150, 53, 57, 62, 64, 66, 67, 69, 71, 73, 76, 77, 79, 84, 86, 87, 92, 101, 102, 103, 104, 105, 106, 108, 109, 110, 111, 112, 113, 115, 118, 119, 120, 121, 122, 123, 124, 126, 127}
NOT
sepal_length,5.3..7.9  Set to be NULL
sepal_length,5.3..7.9 also set to be NULL
Intersection set :  {128, 129, 130, 131, 132, 134, 135, 136, 137, 138, 139, 140, 143, 144, 145, 146, 147, 148, 149, 150, 53, 57, 62, 64, 66, 67, 69, 71, 73, 76, 77, 79, 84, 86, 87, 92, 101, 102, 103, 104, 105, 106, 108, 109, 110, 111, 112, 113, 115, 118, 119, 120, 121, 122, 123, 124, 126, 127}
NOT
sepal_length,5.2..7.9  Set to be NULL
sepal_length,5.2..7.9 also set to be NULL
sepal_length,5.2..7.9 also set to be NULL
Intersection set :  {128, 129, 130, 131, 132, 134, 135, 136, 137, 138, 139, 140, 143, 144, 145, 146, 147, 148, 149, 150, 53, 57, 62, 64, 66, 67, 69, 71, 73, 76, 77, 79, 84, 86, 87, 92, 101, 102, 103, 

petal_length,4.8..6.9 also set to be NULL
petal_length,4.8..6.9 also set to be NULL
Intersection set :  {128, 129, 130, 131, 135, 136, 137, 138, 139, 140, 143, 144, 145, 146, 147, 148, 149, 150, 53, 71, 73, 84, 101, 102, 104, 106, 108, 109, 111, 113, 118, 120, 121, 123, 124, 127}
NOT
petal_length,4.7..6.9  Set to be NULL
sepal_length,4.7..7.9 also set to be NULL
petal_length,4.7..6.9 also set to be NULL
Intersection set :  {128, 129, 130, 131, 135, 136, 137, 138, 139, 140, 143, 144, 145, 146, 147, 148, 149, 150, 53, 71, 73, 84, 101, 102, 104, 106, 108, 109, 111, 113, 118, 121, 123, 124, 127}
NOT
sepal_width,2.5..4.4  Set to be NULL
sepal_width,2.5..4.4 also set to be NULL
sepal_width,2.5..4.4 also set to be NULL
petal_length,2.5..6.9 also set to be NULL
Intersection set :  {128, 129, 130, 131, 135, 136, 137, 138, 139, 140, 143, 144, 145, 146, 147, 148, 149, 150, 53, 71, 73, 84, 101, 102, 104, 106, 108, 109, 111, 113, 118, 121, 123, 124, 127}
NOT
sepal_width,2.3..4.4  Set to be NULL
sep

petal_width,0.6..2.5 also set to be NULL
Intersection set :  {128, 129, 130, 131, 132, 134, 135, 136, 137, 138, 139, 140, 143, 144, 145, 146, 147, 148, 149, 150, 53, 57, 62, 64, 66, 67, 69, 71, 73, 76, 77, 79, 84, 86, 87, 92, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 115, 118, 119, 120, 121, 122, 123, 124, 126, 127}
NOT
petal_width,0.5..2.5  Set to be NULL
petal_width,0.5..2.5 also set to be NULL
Intersection set :  {128, 129, 130, 131, 132, 134, 135, 136, 137, 138, 139, 140, 143, 144, 145, 146, 147, 148, 149, 150, 53, 57, 62, 64, 66, 67, 69, 71, 73, 76, 77, 79, 84, 86, 87, 92, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 115, 118, 119, 120, 121, 122, 123, 124, 126, 127}
NOT
petal_width,0.3..2.5  Set to be NULL
petal_width,0.3..2.5 also set to be NULL
Intersection set :  {128, 129, 130, 131, 132, 134, 135, 136, 137, 138, 139, 140, 143, 144, 145, 146, 147, 148, 149, 150, 53, 57, 62, 64, 66, 67, 69, 71, 73, 76, 77, 79, 84, 86, 87, 92, 101, 102, 

sepal_width,2.3..4.4 also set to be NULL
Intersection set :  {128, 129, 130, 131, 136, 137, 138, 139, 140, 143, 144, 145, 146, 147, 148, 149, 150, 71, 84, 101, 102, 104, 106, 108, 109, 111, 113, 118, 121, 123, 124, 127}
NOT
petal_width,1.6..2.5  Set to be NULL
petal_length,1.6..6.9 also set to be NULL
petal_length,1.6..6.9 also set to be NULL
petal_width,1.6..2.5 also set to be NULL
petal_width,1.6..2.5 also set to be NULL
Intersection set :  {128, 129, 130, 131, 136, 137, 138, 139, 140, 143, 144, 145, 146, 147, 148, 149, 150, 71, 84, 101, 102, 104, 106, 108, 109, 111, 113, 121, 123, 124, 127}
NOT
sepal_width,2.2..3.7  Set to be NULL
sepal_width,2.2..4.4 also set to be NULL
sepal_width,2.2..3.7 also set to be NULL
sepal_width,2.2..3.8 also set to be NULL
sepal_width,2.2..3.8 also set to be NULL
sepal_width,2.2..4.0 also set to be NULL
sepal_width,2.2..4.0 also set to be NULL
sepal_width,2.2..4.2 also set to be NULL
sepal_width,2.2..4.3 also set to be NULL
Intersection set :  {129, 130,

petal_length,5.3..6.9 also set to be NULL
Intersection set :  {129}
NOT
petal_length,5.2..6.9  Set to be NULL
sepal_length,5.2..7.9 also set to be NULL
sepal_length,5.2..7.9 also set to be NULL
petal_length,5.2..6.9 also set to be NULL
petal_length,5.2..6.9 also set to be NULL
Intersection set :  set()


In [61]:
print("Time to run the algorithm (for Upper Approximation): ", round(elapsed_time, 3), "Sec")
print("\n")
print(*final_rules, sep='\n')

with open('../data/Iris/Rules/Iris_10_lost_upper.txt', 'w') as f:
    for item in final_rules:
        f.write("%s\n" % item)

Time to run the algorithm (for Upper Approximation):  10.877 Sec


['(sepal_length,4.3..5.8) & (petal_length,1.0..2.5) -> (class,Iris-setosa)', '(petal_length,1.0..1.8) & (sepal_length,4.8..7.9) -> (class,Iris-setosa)']
['(sepal_length,5.0..7.9) & (petal_length,3.7..4.7) -> (class,Iris-versicolor)', '(petal_length,4.5..6.9) & (petal_width,0.1..1.2) -> (class,Iris-versicolor)']
['(sepal_length,6.0..7.9) & (petal_width,1.8..2.5) -> (class,Iris-viginica)', '(petal_length,5.2..6.9) & (sepal_length,6.5..7.9) -> (class,Iris-viginica)']
